In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import statsmodels.api as sm
from statsmodels.formula.api import ols
from sklearn.model_selection import cross_val_score, train_test_split
import numpy as np
import warnings
warnings.filterwarnings('ignore')


pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

In [2]:
sales = pd.read_csv("kc_house_data_train.csv", index_col = 0)
holdout = pd.read_csv("kc_house_data_test_features.csv", index_col = 0)
filledrooms = pd.read_csv("bedroomsfilled.csv", index_col = 0)
extra = pd.read_csv("zip_code_extra_data.csv", index_col = 0)

In [3]:
# def filled_data(df,df2):
#     index=df.loc[df.bedrooms == 0].index.to_list()
#     df.drop(df.index[[index]], inplace=True)
#     frames= [df, df2]
#     train = pd.concat(frames)
#     return train
# sales=filled_data(sales,filledrooms)

In [4]:
print(sales.shape)
sales.head()

(17290, 21)


,id,date,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,condition,grade,sqft_above,sqft_basement,yr_built,yr_renovated,zipcode,lat,long,sqft_living15,sqft_lot15
0,2591820310,20141006T000000,365000.0,4,2.25,2070,8893,2.0,0,0,4,8,2070,0,1986,0,98058,47.4388,-122.162,2390,7700
1,7974200820,20140821T000000,865000.0,5,3.00,2900,6730,1.0,0,0,5,8,1830,1070,1977,0,98115,47.6784,-122.285,2370,6283
2,7701450110,20140815T000000,1038000.0,4,2.50,3770,10893,2.0,0,2,3,11,3770,0,1997,0,98006,47.5646,-122.129,3710,9685
3,9522300010,20150331T000000,1490000.0,3,3.50,4560,14608,2.0,0,2,3,12,4560,0,1990,0,98034,47.6995,-122.228,4050,14226
4,9510861140,20140714T000000,711000.0,3,2.50,2550,5376,2.0,0,0,3,9,2550,0,2004,0,98052,47.6647,-122.083,2250,4050


In [5]:
extra.reset_index(inplace = True)

In [6]:
extra["zipcode"].nunique()

70

In [7]:
#sales[sales["zipcode"]==98116]

In [8]:
# sales["month"] = pd.DatetimeIndex(sales["date"]).month
# sales.drop(columns=["id","date"], inplace = True)

In [9]:
# df=sales.merge(extra, how="right", on="zipcode")
#extra.set_index('zipcode', inplace=True)
training_data = pd.merge(sales, extra, how='right', on='zipcode')

In [10]:
sales["zipcode"].nunique()

70

In [11]:
training_data.shape

(17755, 75)

In [13]:
extra.head()

,zipcode,full_time_earnings,part_time_earnings,no_earnings,car_truck_van,public_transportation,taxi,motorcycle,bike_walk_other,work_home,less_than_hs,high_school_grad,associates,bachelors,masters,professional_school_degree,doctorate,public_school_enrollment,private_school_enrollment,no_school_enrollment,for_rent,rented_and_unoccupied,for_sale_only,sold_and_unoccupied,seasonal_or_rec_use,migrant_worker_housing,vacant_other_reasons,in_occupied_housing_units,adult_correctional_facility,juvenile_facilities,nursing_facilities,other_institutional,military_quarters,other_noninstitutional,house_owned_with_mortgage,house_owned_outright,renter_occupied_housing,vacant_households,husband_and_wife_house,single_guardian,singles,singles_with_roommate,male,female,median_age,male_median_age,female_median_age,population,population_density_per_sq_mile,total_housing_units,median_home_value,land_area,water_area_sq_mile,occupied_housing_units,median_household_income
0,98001,11210,7226,8178,14452,1057,0,54,212,579,1957,12586,3015,3655,942,152,72,5979,470,800,146,10,146,21,33,0,176,31694,30,0,4,21,0,162,6975,1369,2373,532,6468,1772,1801,676,16047,15864,38,38,38,31911,1717,11249,268800.0,18.59,0.32,10717,71746.0
1,98002,9534,7439,9023,12319,986,84,65,618,361,3924,13231,1922,1968,587,149,20,5325,243,1112,565,12,189,28,38,0,153,31247,0,10,215,0,0,175,3970,1992,6533,985,4665,2772,3924,1134,15564,16083,35,34,36,31647,4423,13480,167100.0,7.15,0.13,12495,43568.0
2,98003,13989,8908,12669,17082,1386,0,77,784,944,3570,15997,3049,4438,1212,296,156,6956,601,1446,631,25,197,39,67,0,227,43511,3,0,421,0,0,216,6313,2167,8770,1186,6863,3610,5395,1382,21343,22808,35,33,36,44151,3800,18436,221600.0,11.62,0.11,17250,45894.0
3,98004,11548,5699,7890,10628,1383,11,32,1748,1454,653,5275,1384,8945,4170,1253,715,2904,1385,462,1383,141,1111,60,361,1,191,27758,0,0,50,0,0,138,4536,2118,6666,3248,5946,1129,5142,1103,13894,14052,40,38,41,27946,3977,16568,723300.0,7.03,1.61,13320,93521.0
4,98005,6500,3307,4636,6461,1032,27,63,389,524,454,2954,875,4743,2368,500,412,2148,566,185,187,13,89,29,140,0,73,17549,0,0,0,0,0,165,3193,1168,3178,531,3800,781,2286,672,8930,8784,38,36,40,17714,2361,8070,568700.0,7.50,0.01,7539,84774.0


In [14]:
extra.tail()

,zipcode,full_time_earnings,part_time_earnings,no_earnings,car_truck_van,public_transportation,taxi,motorcycle,bike_walk_other,work_home,less_than_hs,high_school_grad,associates,bachelors,masters,professional_school_degree,doctorate,public_school_enrollment,private_school_enrollment,no_school_enrollment,for_rent,rented_and_unoccupied,for_sale_only,sold_and_unoccupied,seasonal_or_rec_use,migrant_worker_housing,vacant_other_reasons,in_occupied_housing_units,adult_correctional_facility,juvenile_facilities,nursing_facilities,other_institutional,military_quarters,other_noninstitutional,house_owned_with_mortgage,house_owned_outright,renter_occupied_housing,vacant_households,husband_and_wife_house,single_guardian,singles,singles_with_roommate,male,female,median_age,male_median_age,female_median_age,population,population_density_per_sq_mile,total_housing_units,median_home_value,land_area,water_area_sq_mile,occupied_housing_units,median_household_income
66,98178,8460,4904,6333,9818,1062,0,44,359,468,2794,8477,1386,3334,819,399,115,3324,525,520,256,8,117,33,33,0,175,23939,0,0,4,0,0,149,4332,1340,2996,622,3629,2091,2256,692,11857,12235,38,37,39,24092,4966,9290,283100.0,4.85,1.02,8668,58707.0
67,98188,7143,5522,6189,8998,1248,0,3,539,299,2141,9020,1382,2302,639,115,99,3221,238,571,371,24,111,12,51,0,127,23034,0,0,11,0,0,66,3088,1335,4840,696,3378,1867,3108,910,11866,11245,35,35,36,23111,3062,9959,226800.0,7.55,0.29,9263,40815.0
68,98198,11245,7005,10280,13694,1325,0,47,346,333,4043,12572,2243,3773,1177,255,137,4583,606,692,568,40,285,29,106,0,194,33121,831,0,433,0,0,199,5705,1799,5602,1222,5589,2532,3944,1041,17232,17352,38,36,39,34584,4441,14328,258500.0,7.79,0.90,13106,56364.0
69,98199,8422,4458,3964,8276,1622,0,52,749,680,353,3554,1087,6447,2387,1042,599,2112,999,216,240,18,154,38,104,0,158,19620,0,0,12,5,0,49,4381,1557,3160,712,4182,817,3010,1089,9607,10079,42,41,42,19686,4714,9810,606200.0,4.18,1.50,9098,82882.0
70,98116,9304,5177,5137,9414,1934,0,33,693,924,480,5516,1656,6763,2509,837,430,2011,1322,433,265,21,190,38,143,0,200,22018,0,0,185,0,0,38,4964,1496,4644,857,4165,994,4593,1352,10640,11601,42,41,43,22241,7523,11961,467900.0,2.96,0.73,11104,74274.0
